In [1]:
# new data structure for STOPWATCHER
# build up tables BY VEHICLE - a history of observations of each vehicle
# then can go and reconstruct unique history for each VEHICLE, based on last observeration before it STOPPED APPROACHING each stop in the _stoplist_

In [2]:
import argparse


# relative import
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from lib.reportcard_helpers import *

In [3]:
source = 'nj'
route = 87

In [4]:
(conn, db) = db_setup(route)

In [5]:
# only get buses APPROACHING A STOP
arrival_query = ('SELECT * FROM stop_predictions \
                WHERE (rd = %s AND pt = "APPROACHING") \
                ORDER BY timestamp;' % route)

In [6]:
df = pd.read_sql_query(arrival_query, conn)

In [7]:
df = timestamp_fix(df)

In [8]:
# diganostic
# plucking out unique vehicle-stop pair (should be 2-3 times per day on a full history?)
df_87_21058 = df.loc[(df.rd == '87') & (df.stop_id=='21058')]
df_87_21058.head(3)

,pkey,cars,consist,fd,m,name,pt,rd,rn,scheduled,stop_id,stop_name,v,timestamp
timestamp,,,,,,,,,,,,,,
2018-05-30 18:25:50,71,,,87 HOBOKEN-PATH VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,21058,PALISADE AVE + FRANKLIN ST,5741,2018-05-30 18:25:50
2018-06-04 16:49:15,22257,,,87 HOBOKEN-PATH VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,21058,PALISADE AVE + FRANKLIN ST,5704,2018-06-04 16:49:15
2018-06-04 16:59:07,28897,,,87 HOBOKEN-PATH VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,21058,PALISADE AVE + FRANKLIN ST,5710,2018-06-04 16:59:07


In [9]:
# put it all together

# 1 group all arrival predictions by unique vehicle-stop combinations
df.groupby(['v','stop_id'])

# 2 gets the keys for unique vehicle-stop combinations
# a list of tuples
# [(u'5403', u'20640'), (u'5704', u'20933'), .....

keys = df.groupby(['v','stop_id']).groups.keys()

# 3
# loop over list of unique vehicle-stop combinations
# make a master list of stop history for each vehicle
# each row in history is a pandas series

history=[]

for i,k in enumerate(keys):
   
    # per vehicle history of all observations APPROACHING stops
    v = k[0]
    stop_id = k[1]
    # print v,stop_id
    
    # get the rows for this vehicle-stop
    for index,row in df.iterrows():
        if ( row['v']== v ) and ( row['stop_id']== stop_id):
            history.append(row)
        else:
            pass
    
    

In [26]:
# diagnostic
print type(history[0])
print
print history[0]
print
print history[0]['fd']


<class 'pandas.core.series.Series'>

pkey                                9914
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                      StopPrediction
pt                           APPROACHING
rd                                    87
rn                                    87
scheduled                          false
stop_id                            20640
stop_name       CENTRAL AVE + BEACON AVE
v                                   5403
timestamp            2018-06-01 19:09:33
Name: 2018-06-01 19:09:33, dtype: object

87 GATES AVE VIA JOURNAL SQ


In [39]:
# convert history back to a df?

df_final = pd.concat([x for x in history],axis=1)
df_final.T

,pkey,cars,consist,fd,m,name,pt,rd,rn,scheduled,stop_id,stop_name,v,timestamp
2018-06-01 19:09:33,9914,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20640,CENTRAL AVE + BEACON AVE,5403,2018-06-01 19:09:33
2018-05-30 18:30:44,2028,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,32216,BERGEN AVE + ACADEMY ST,5398,2018-05-30 18:30:44
2018-05-30 18:30:48,2255,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,32216,BERGEN AVE + ACADEMY ST,5398,2018-05-30 18:30:48
2018-06-04 17:04:07,32305,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,21044,OLD BERGEN RD + WINFIELD AVE,5895,2018-06-04 17:04:07
2018-06-04 17:04:11,32584,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,21044,OLD BERGEN RD + WINFIELD AVE,5895,2018-06-04 17:04:11
2018-06-04 16:53:02,24548,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20940,MONTICELLO AVE + JEWETT AVE,5247,2018-06-04 16:53:02
2018-06-04 16:53:13,25023,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20940,MONTICELLO AVE + JEWETT AVE,5247,2018-06-04 16:53:13
2018-06-04 17:09:02,35153,,,87 HOBOKEN-PATH VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20685,MARTIN LUTHER KING JR DR + KEARNEY AVE,5733,2018-06-04 17:09:02
2018-05-30 18:26:56,841,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20692,DR. M.L. KING DR + FORREST ST,5705,2018-05-30 18:26:56
2018-05-30 18:50:05,7021,,,87 GATES AVE VIA JOURNAL SQ,1,StopPrediction,APPROACHING,87,87,false,20701,MARTIN LUTHER KING JR DR + STEVENS AVE,5398,2018-05-30 18:50:05
